# Practice: Spotify's API

This notebook contains a set of exercises that will guide you through the different steps of this practice exercise. Devise solutions that are code-based, i.e. not hard-coded or manually computed. 

<div class="alert alert-success">The aim of this exercise is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API.</div>

## Getting client credentials

Spotify's API uses an Authentication scheme called OAuth. Hence, before starting to make requests, you need to get your client credentials to the Spotify API. 

To do so, you need to have a Spotify account (free or paid). If you don't have one yet, please create a free account before moving on. Once you do, head over to Spotify for Developers, open your [Dashboard](https://developer.spotify.com/dashboard/) and log in with your account. 

Click on “CREATE APP”. Fill in the name and description for your project. Under "Redirect URI", enter "http://localhost:8888/callback" (don't worry about this, as it won't be used). Finally, it the tick boxes, select "Web API". 

Once your App has been created, you should see the app name in the Dashboard. Click on it, then on "Settings", and then on "View Client Secret".  The numbers for “Client ID” and “Client Secret” on correspond to your client credentials.

<div class="alert alert-info">Create two new variables, <b>client_id</b> and <b>client_secret</b>, that store your ID and Key, respectively</div>

In [243]:
client_id = 'e9a23f0ca78341a7bab3b93c79149256'
client_secret = 'a81fea09f24a41dea9ed058a5f0d82c0'

These are the keys that you will use throughout the remaining of the assignment.



Great! We are good to go. Next step is getting an access token.

## Getting an access token / key

In order to access the various endpoints of the Spotify API, we need to pass an access token. 

To get one, we need to pass a ```POST``` request with our client credentials. This request will create a token resource in the server and respond back with it. We can build this ```POST``` request using ```requests``` library. In other words, the first API call in the spotify API is the call to get the access token. 

<div class="alert alert-info">Run the following cell to built your POST request</div>

In [244]:
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

# POST the request
response =  response = requests.post(auth_url, data=params)
response.json()

{'access_token': 'BQC1jSCtQln7x6tqyNfETLcyJtmSl9L5yTSBtSoOEZv4ZWfeLIp5bjOxrh_U8saadW58eaxvcslicWnGz40pqeMqQZrY58efDmjsOH1rXB1jN0PF_XA',
 'token_type': 'Bearer',
 'expires_in': 3600}

Take a look at the response you obtained. The access_token you just retrieved will expire after one hour.

<div class="alert alert-info">Retrieve your token from <b>auth_response</b> and save it in a new variable called <b>access_token</b>. </div>

<div class="alert alert-warning">Make sure you define the <i>access_token</i> variable such that it will be updated each time your code is run from scratch, i.e. make sure it hasn't expired by the time your code is graded.</div>

In [245]:
# Check if the request was successful
if response.status_code == 200:
    # Retrieve the access token
    auth_response = response.json()
    access_token = auth_response['access_token']
else:
    access_token = None  # Handle the error case

access_token

'BQC1jSCtQln7x6tqyNfETLcyJtmSl9L5yTSBtSoOEZv4ZWfeLIp5bjOxrh_U8saadW58eaxvcslicWnGz40pqeMqQZrY58efDmjsOH1rXB1jN0PF_XA'

As above, the access token will be used throughout the remaining of the assignment.



This token is your golden ticket to access Spotify's API. A copy of this string is now stored in the server, so that everytime you make a request to the API the server will check that the token you provide and the one it has in store match.

<img src="https://www.dropbox.com/s/hgb02k4h1mtdv22/header.png?raw=1" width="500">

Similar to the Cohere API, Spotify's API expects you to include your access token in the requests header. We won't use headers in this course and this is the only exception: when some APIs require to specify the authentication in the header. Hence, the header has already been formatted for you. 

<div class="alert alert-info">Run the following cell to save the header in a new variable so that you can use it later on.</div>

In [246]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

## Poking around

Spotify's API provides numerous endpoints to access things like album listings, artist information, playlists, even Spotify-generated audio analysis of individual tracks, which include their time signature or measurements such as their “danceability” or "loudness". You can take a look at all the information available by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). In this assignment you will use several of these endpoints.

In order to get a feel of how the API works, we will begin by making a ```GET``` request to the ```audio-features``` endpoint to extract data for a specific track. In particular, let's retrieve all the information for Radiohead's *Creep* song. 

The first thing you need is to identify the appropriate URL or path to direct your request to. The urls for all Spotify API endpoints follow the same structure. They all use the base URL for the API and are then defined as a concatenation of ```base_url + endpoint```. Sometimes, you will also need to provide some additional information as part of the URL. In the case of ```audio-features```, however, it is enough with just the ```base_url``` and the ```endpoint``` name.

The ```base_url``` is defined below:

In [247]:
base_url = 'https://api.spotify.com/v1'

<div class="alert alert-info">Define the url for the audio-features endpoint by following the instructions above. Store it in a variable called <b>audio_features_endpoint</b>.</div>

In [248]:
audio_features_endpoint = f'{base_url}/audio-features'

audio_features_endpoint

'https://api.spotify.com/v1/audio-features'

Next thing we need is to fill in the request body. If you check the documentation you'll see that the ```audio-features``` endpoint takes the following query parameters.

<img src="https://www.dropbox.com/s/s4zs6wlue0u16cu/body.png?raw=1" width="500">

Hence, the final thing you need to extract data about Radiohead's Creep song is to locate its ```id```. This is its unique identifier. Spotify has unique ids for tracks, for artists, for albums, for playlists, etc.

![Creep](https://www.dropbox.com/s/kufj6ww2yn069gb/creep.png?raw=1)

You can get the ```id``` for any song by going to Spotify, looking for the song, clicking the “…” by the song name, then “Share” and then “Copy Spotify URI”. 

<div class="alert alert-warning">Note that this procedure also works for retrieving ids for artists, albums or any other resource type. Alternatively, you can also retrieve the id for a song or a playlist by clicking "..." by the name, then "Share" and the "Copy Link". This will return a full link to the resource. The id corresponds to the string right after the last slash sign / and before the ? sign</div>

This URI should be a string that includes something like **spotify:track:**, followed by an alphanumeric sequence. This sequence is the ID you are looking for.

<div class="alert alert-info">Create a new variable called <b>track_id</b> that stores the ID for Radiohead's song Creep.</div>

In [249]:
track_id = '70LcF31zb1H0PyJoS1Sx1r'

Now that you have the id, let's format the URL of the request and send the actual GET request to retrieve the data:

In [250]:
# Format the URL for the GET request
request_url = f'{audio_features_endpoint}/{track_id}'

# Send the GET request to retrieve the data
response = requests.get(request_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Retrieve the audio features data
    audio_features_data = response.json()
    print(audio_features_data)  # Print the data for verification
else:
    print("Error retrieving audio features:", response.status_code, response.text)

{'danceability': 0.515, 'energy': 0.43, 'key': 7, 'loudness': -9.935, 'mode': 1, 'speechiness': 0.0372, 'acousticness': 0.0097, 'instrumentalness': 0.000133, 'liveness': 0.129, 'valence': 0.104, 'tempo': 91.844, 'type': 'audio_features', 'id': '70LcF31zb1H0PyJoS1Sx1r', 'uri': 'spotify:track:70LcF31zb1H0PyJoS1Sx1r', 'track_href': 'https://api.spotify.com/v1/tracks/70LcF31zb1H0PyJoS1Sx1r', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70LcF31zb1H0PyJoS1Sx1r', 'duration_ms': 238640, 'time_signature': 4}


Now that everything is ready, you can send the actual GET request to retrieve the data.

<div class="alert alert-info">Remember to pass the <i>url</i>, the <i>headers</i> and the <i>params</i> dictionary as arguments to the <i>get</i> function. Convert the response to <i>json</i> format and store it in a new variable called <b>creep</b>.</div>

In [251]:
# Define any parameters if needed (empty in this case)
params = {}

# Send the GET request to retrieve the data
creep = requests.get(request_url, headers=headers, params=params).json()

# Print the retrieved data for verification
print(creep)

{'danceability': 0.515, 'energy': 0.43, 'key': 7, 'loudness': -9.935, 'mode': 1, 'speechiness': 0.0372, 'acousticness': 0.0097, 'instrumentalness': 0.000133, 'liveness': 0.129, 'valence': 0.104, 'tempo': 91.844, 'type': 'audio_features', 'id': '70LcF31zb1H0PyJoS1Sx1r', 'uri': 'spotify:track:70LcF31zb1H0PyJoS1Sx1r', 'track_href': 'https://api.spotify.com/v1/tracks/70LcF31zb1H0PyJoS1Sx1r', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70LcF31zb1H0PyJoS1Sx1r', 'duration_ms': 238640, 'time_signature': 4}



<div class="alert alert-warning">Often Spotify has several instances of a track in its catalogue, each available in a different set of markets. This commonly happens when the track the album is on has been released multiple times under different licenses in different markets. These tracks are linked together so that when a user tries to play a track that isn’t available in their own market, the Spotify mobile, desktop, and web players try to play another instance of the track that is available in the user’s market. Even if this issue won't prevent you from being able to play the  song in your device, it might result in different ids for the different markets. Note that the test below is written taking the values for the Spanish market into account.</div>

<div class="alert alert-warning">Note that you can specify the market for this and for future queries through the <i>market</i> parameter.</div>

Congrats! You just made your first successful request to Spotify's API! Feel free to take a look at the information included in the response. Pay special attention to the way in which information is presented. Once you are done, let's move on to some actual work!

## Getting data from a playlist

In the following exercise you will build a dataset containing data about different songs. You can either use a playlist of your own, or use the one mentioned in the documentation.


<div class="alert alert-info">Create a variable called <b>playlist_id</b> that stores the id of your playlist of choice.</div>

<div class="alert alert-warning">Your <i>playlist_id</i> should contain only alphanumeric characters. This means that characters such as ? $ % & / ! should not be included.</div>

In [252]:
playlist_id = '3xklMkftvzK5xCTlNZXNZA'

In the following you are going to extract information about the different tracks included in this playlist. Hence, let's make sure that the variabl ei spropelry created and that it refers to an actual id.

The next step will be making a request to get full details of the tracks included in your chosen playlist. Remember that you can take a look at all the information available at the different endpoints in Spotify's API by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). Locate the right endpoint for your query and read the Docs to find out how to build your request. 

<div class="alert alert-info"><b>Exercise 1 </b>Write the code to retrieve all the items from your chosen playlist from the <i>tracks</i> endpoint. When making your request don't use any of the optional arguments. Direct your request to the right endpoint instead. Store the <i>raw</i> response in a new variable called <i>playlist_response</i>.
</div>

<div class="alert alert-warning">To complete this exercise you must use the requests library</div>

In [253]:
# Define the URL for the playlist tracks endpoint
playlist_tracks_endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

# Send the GET request to retrieve the playlist items
playlist_response = requests.get(playlist_tracks_endpoint, headers=headers)

# Verification
print(playlist_response.status_code)  # Should be 200 for a successful request
# print(playlist_response.json())

200


<div class="alert alert-info"><b>Exercise 2 </b>Convert the response to JSON format and store the result in a new variable called <i>playlist</i>.</div>

In [254]:
playlist = playlist_response.json()

playlist

{'href': 'https://api.spotify.com/v1/playlists/3xklMkftvzK5xCTlNZXNZA/tracks?offset=0&limit=100',
 'items': [{'added_at': '2020-10-03T06:10:05Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/8fwj9wkx5ekmrxrrm0sdp53kq'},
    'href': 'https://api.spotify.com/v1/users/8fwj9wkx5ekmrxrrm0sdp53kq',
    'id': '8fwj9wkx5ekmrxrrm0sdp53kq',
    'type': 'user',
    'uri': 'spotify:user:8fwj9wkx5ekmrxrrm0sdp53kq'},
   'is_local': False,
   'primary_color': None,
   'track': {'preview_url': 'https://p.scdn.co/mp3-preview/0b50c60d26726c8a718d5aaf70bdd1496181f595?cid=e9a23f0ca78341a7bab3b93c79149256',
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
  

The following cells contain the tests that will grade your code. **Don't modify them**. Simply leave them as they are.

Take your time to familiarize yourself with the data and how they are presented. Note that, by default, Spotify's API only returns information about a maximum of 100 tracks in a playlist. If your playlist of choice has more that 100 tracks, you'll retrieve the data only for the first 100 of them.

<div class="alert alert-danger">Throghout the following exercises you may come across data that are missing. If so, encode these data using a <b>None</b> (Nonetype), unless otherwise stated. </div>

## Retrieving basic track information

In what follows, you are going to retrieve specific data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 3 </b>Write the code to extract the title (in string form), the name of the album (in string form), the name of the artist (in string form), the duration (in integer form), the track number (in integer form), the popularity (in integer form), and the id (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>title</i>, <i>album</i>, <i>artist</i>, <i>duration</i>, <i>track_number</i>, <i>track_popularity</i>, and <i>track_id</i>, respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.<br></div>

<div class="alert alert-warning">Make sure you correctly set all variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

In [255]:
# Step 1: Initialize empty lists to store the extracted data from the playlist.
# This will help us organize the information for each track clearly.

title = []              # List to store track titles
album = []              # List to store album names
artist = []             # List to store artist names
duration = []           # List to store track durations in milliseconds
track_number = []       # List to store track numbers
track_popularity = []   # List to store track popularity scores
track_id = []           # List to store track IDs

# Step 2: Loop through each item in the playlist to extract track details.
# The items are accessed from the 'playlist' variable we created earlier.
for item in playlist['items']:
    # We extract the track information from each item.
    track = item['track']  # Each item has a 'track' key that contains the track details.

    # Step 3: Append the necessary data to the lists, ensuring to handle missing data.
    # Using conditional expressions to avoid KeyErrors and ensure we follow best practices.
    
    # Extract track title
    title.append(track['name'] if track and 'name' in track else None)
    
    # Extract album name
    album.append(track['album']['name'] if track and 'album' in track and 'name' in track['album'] else None)
    
    # Extract artist name (only the first artist)
    artist.append(track['artists'][0]['name'] if track and 'artists' in track and len(track['artists']) > 0 else None)
    
    # Extract track duration (in milliseconds)
    duration.append(track['duration_ms'] if track and 'duration_ms' in track else None)
    
    # Extract track number
    track_number.append(track['track_number'] if track and 'track_number' in track else None)
    
    # Extract track popularity score
    track_popularity.append(track['popularity'] if track and 'popularity' in track else None)
    
    # Extract track ID
    track_id.append(track['id'] if track and 'id' in track else None)

# Step 4: Verifying the extracted data.
print("Track Titles:", title)
print("Album Names:", album)
print("Artist Names:", artist)
print("Track Durations (ms):", duration)
print("Track Numbers:", track_number)
print("Track Popularity:", track_popularity)
print("Track IDs:", track_id)


Track Titles: ["Tell Me Why I'm Waiting", 'Before You Go', 'Hold Me While You Wait', 'Falling', 'Waves', 'Be Alright', 'Alag Aasmaan', 'Apocalypse', "Baby You're Worth It", 'Trying My Best', 'Ghost Of You', 'Creep', 'Mirrors', 'Yellow Lights', 'death bed (coffee for your head)', "I've Changed for You", "Can't Take My Eyes off You", 'You & Me (The Wildfire)', 'Chasing Cars', 'Like U', 'NUMB', 'Imagination', 'I Was Wrong', 'All I Want', 'High Hopes', 'The One', 'Next To Me', 'Everglow', 'All Through the Night', 'Make It Without You', 'distance', 'Another Love', 'Baarishein', 'Riha', 'You Are The Reason', 'my dear, i will think of you', 'Wake Me up When September Ends', 'Yellow', "Say You Won't Let Go", 'Moral of the Story', 'Can I Be Him', 'The Night We Met', 'Falling Like The Stars', 'Say Something', 'I Found', 'Jocelyn Flores', "Don't Forget Me", 'Forever', 'Dead Eyes', 'The Wisp Sings', 'Dancing With Your Ghost', 'If By Chance']
Album Names: ["Tell Me Why I'm Waiting", 'Before You Go'

<div class="alert alert-info"><b>Exercise 4 </b>Write the code to extract the number of available markets (in integer form) for all the tracks included in your chosen playlist. Store these data in a separate list called <i>n_available_markets</i>. As above, entries in this list should appear in the same order as they are presented in the playlist.</div>

In [256]:
# Step 1: Initialize an empty list to store the number of available markets for each track.
n_available_markets = []  # This list will hold the market counts in the same order as the tracks.

# Step 2: Loop through each item in the playlist to extract the number of available markets.
for item in playlist['items']:
    track = item['track']  # Extract the track information from the current item.

    # Step 3: Append the number of available markets to the list.
    # We use the 'available_markets' key to get the list of markets and calculate its length.
    n_available_markets.append(len(track['available_markets']) if track and 'available_markets' in track else None)

# Step 4: Verifying the extracted number of available markets.
print("Number of Available Markets:", n_available_markets)

Number of Available Markets: [177, 184, 184, 185, 184, 184, 0, 183, 0, 0, 179, 0, 185, 179, 185, 185, 0, 185, 0, 0, 182, 0, 185, 185, 185, 185, 184, 184, 185, 176, 120, 184, 0, 0, 184, 0, 185, 185, 173, 185, 173, 182, 185, 182, 185, 185, 0, 184, 185, 0, 185, 185]


Each track in the playlist is associated to an album. Let's retrieve the corresponding release date.

<div class="alert alert-info"><b>Exercise 5 </b>Write the code to extract the release year (in int form) and month (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>release_year</i> and <i>release_month</i>, respectively. Entries should appear in these lists in the same order as they are presented in the playlist.</div>

<div class="alert alert-warning">Make sure you correctly set the variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

<div class="alert alert-warning">Months should be stored with their full names and capitalized: use <i>September</i> for 09, <i>January</i> for 01, etc.</div>

In [257]:
# Step 1: Initialize empty lists to store release years and release months.
release_year = []  # This list will hold the release years of each track.
release_month = []  # This list will hold the release months of each track.

# Step 2: Define a mapping of month numbers to their full names.
month_names = {
    '01': 'January', '02': 'February', '03': 'March', '04': 'April',
    '05': 'May', '06': 'June', '07': 'July', '08': 'August',
    '09': 'September', '10': 'October', '11': 'November', '12': 'December'
}

# Step 3: Loop through each item in the playlist to extract release dates.
for item in playlist['items']:
    track = item['track']  # Extract the track information.

    # Step 4: Retrieve the album release date.
    release_date = track['album']['release_date'] if track and 'album' in track else None

    # Step 5: Extract the year and month from the release date.
    if release_date:  # Ensure release_date is not None
        year, month = release_date.split('-')[:2]  # Split the date string and get year and month.
        release_year.append(int(year))  # Append year as an integer.
        release_month.append(month_names[month])  # Append month using the mapping.
    else:
        # If release_date is None, append None to both lists to maintain alignment.
        release_year.append(None)
        release_month.append(None)

# Step 6: Verifying the extracted release years and months.
print("Release Years:", release_year)
print("Release Months:", release_month)

Release Years: [2020, 2019, 2019, 2019, 2019, 2018, 2020, 2017, 2018, 2019, 2018, 1993, 2013, 2018, 2020, 2018, 2019, 2019, 2006, 2019, 2018, 2018, 2010, 2013, 2013, 2015, 2017, 2015, 2014, 2010, 2011, 2013, 2018, 2019, 2018, 2020, 2004, 2000, 2016, 2019, 2016, 2015, 2019, 2014, 2015, 2017, 2018, 2019, 2019, 2013, 2019, 2017]
Release Months: ['May', 'November', 'May', 'December', 'March', 'June', 'July', 'June', 'June', 'December', 'June', 'February', 'March', 'March', 'February', 'July', 'November', 'March', 'January', 'November', 'March', 'June', 'June', 'June', 'June', 'February', 'June', 'December', 'March', 'February', 'May', 'June', 'January', 'July', 'March', 'January', 'September', 'July', 'October', 'February', 'October', 'April', 'October', 'January', 'April', 'August', 'November', 'May', 'November', 'November', 'June', 'May']


## Retrieving additional track information

In what follows, you are going to retrieve additional data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 6 </b>Write the code to extract data about the danceability, energy, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence and tempo for all the tracks included in your chosen playlist. Store these data in separate lists called <i>danceability</i> (in float form), <i>energy</i>, <i>loudness</i> (in float form), <i>mode</i> (in int form), <i>speechiness</i> (in float form), <i>acousticness</i> (in float form), <i>instrumentalness</i> (in int and float form), <i>liveness</i> (in float form), <i>valence</i> (in float form) and <i>tempo</i> (in float form), respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.</div>

In [258]:
# Step 1: Initialize empty lists to store additional track features.
danceability = []  # To hold danceability values.
energy = []  # To hold energy values.
loudness = []  # To hold loudness values.
mode = []  # To hold mode values (0 or 1).
speechiness = []  # To hold speechiness values.
acousticness = []  # To hold acousticness values.
instrumentalness = []  # To hold instrumentalness values.
liveness = []  # To hold liveness values.
valence = []  # To hold valence values.
tempo = []  # To hold tempo values.

# Step 2: Loop through each item in the playlist to extract additional data.
for item in playlist['items']:
    track = item['track']  # Extract the track information.

    # Step 3: Retrieve the audio features for the track.
    # You would typically use the track's ID to get its audio features.
    track_id = track['id'] if track else None
    if track_id:
        audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'  # Construct the URL for audio features.
        audio_features_response = requests.get(audio_features_url, headers=headers)  # Send a GET request to the audio features endpoint.
        audio_features = audio_features_response.json()  # Convert the response to JSON.

        # Step 4: Extract relevant audio features, ensuring to handle any missing data.
        danceability.append(audio_features['danceability'] if audio_features and 'danceability' in audio_features else None)
        energy.append(audio_features['energy'] if audio_features and 'energy' in audio_features else None)
        loudness.append(audio_features['loudness'] if audio_features and 'loudness' in audio_features else None)
        mode.append(audio_features['mode'] if audio_features and 'mode' in audio_features else None)
        speechiness.append(audio_features['speechiness'] if audio_features and 'speechiness' in audio_features else None)
        acousticness.append(audio_features['acousticness'] if audio_features and 'acousticness' in audio_features else None)
        instrumentalness.append(audio_features['instrumentalness'] if audio_features and 'instrumentalness' in audio_features else None)
        liveness.append(audio_features['liveness'] if audio_features and 'liveness' in audio_features else None)
        valence.append(audio_features['valence'] if audio_features and 'valence' in audio_features else None)
        tempo.append(audio_features['tempo'] if audio_features and 'tempo' in audio_features else None)
    else:
        # If track_id is None, append None to all lists to maintain alignment.
        danceability.append(None)
        energy.append(None)
        loudness.append(None)
        mode.append(None)
        speechiness.append(None)
        acousticness.append(None)
        instrumentalness.append(None)
        liveness.append(None)
        valence.append(None)
        tempo.append(None)

# Step 5: Verifying the extracted audio features.
print("Danceability:", danceability)
print("Energy:", energy)
print("Loudness:", loudness)
print("Mode:", mode)
print("Speechiness:", speechiness)
print("Acousticness:", acousticness)
print("Instrumentalness:", instrumentalness)
print("Liveness:", liveness)
print("Valence:", valence)
print("Tempo:", tempo)

Danceability: [0.545, 0.459, 0.688, 0.567, 0.519, 0.553, 0.699, 0.369, 0.695, 0.561, 0.459, 0.515, 0.574, 0.444, 0.726, 0.644, 0.525, 0.506, 0.563, 0.77, 0.414, 0.816, 0.436, 0.188, 0.488, 0.43, 0.301, 0.534, 0.309, 0.456, 0.4, 0.445, 0.475, 0.389, 0.329, 0.453, 0.546, 0.429, 0.358, 0.572, 0.696, 0.448, 0.552, 0.407, 0.567, 0.872, 0.516, 0.646, 0.719, 0.521, 0.629, 0.405]
Energy: [0.243, 0.575, 0.53, 0.267, 0.731, 0.586, 0.299, 0.468, 0.253, 0.0587, 0.561, 0.43, 0.512, 0.21, 0.431, 0.203, 0.185, 0.24, 0.592, 0.456, 0.416, 0.0965, 0.32, 0.411, 0.487, 0.587, 0.672, 0.445, 0.204, 0.205, 0.445, 0.537, 0.124, 0.148, 0.235, 0.48, 0.814, 0.661, 0.557, 0.406, 0.543, 0.369, 0.347, 0.147, 0.303, 0.391, 0.553, 0.355, 0.651, 0.0889, 0.328, 0.279]
Loudness: [-13.349, -4.858, -5.507, -6.502, -7.192, -6.319, -12.599, -9.013, -15.884, -17.819, -4.819, -9.935, -6.664, -10.811, -8.765, -16.092, -14.938, -14.737, -4.571, -13.867, -8.024, -22.035, -10.405, -9.733, -6.371, -8.003, -4.497, -9.155, -10.867, 

## Retrieving artist information

In what follows, you are going to retrieve data about the artists for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 7 </b>Write the code to extract data about the total number of followers (in int form), the first listed genre (in string form) and the popularity (in int form) for the artists of all the tracks included in your chosen playlist. Store these data in separate lists called <i>artist_followers</i>, <i>genre</i> and <i>artist_popularity</i>. In cases where no genre is given, fill in the corresponding entry using a None.</div>

<div class="alert alert-warning">Note that the artist information has to to be extracted from the tracks themselves.</div>

In [259]:
# Step 1: Create empty lists to hold artist information.
artist_followers = []  # Number of followers for each artist.
genre = []  # First genre for each artist.
artist_popularity = []  # Popularity score for each artist.

# Step 2: Loop through the playlist items to get artist data.
for item in playlist['items']:
    track = item['track']  # Get the track information.

    # Check if the track exists.
    if track:
        artists = track['artists']  # Get the list of artists.

        # Check if there are any artists.
        if artists:
            first_artist = artists[0]  # Get the first artist.

            # Get the artist's ID.
            artist_id = first_artist['id'] if first_artist else None
            
            # Step 3: Make a request to get the artist's information.
            if artist_id:
                artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'  # URL for artist info.
                artist_response = requests.get(artist_url, headers=headers)  # Send GET request.
                artist_info = artist_response.json()  # Convert response to JSON.

                # Step 4: Extract artist details or use None if missing.
                followers = artist_info.get('followers', {}).get('total', None)  # Get total followers.
                first_genre = artist_info['genres'][0] if artist_info.get('genres') else None  # Get first genre.
                popularity = artist_info.get('popularity', None)  # Get popularity.

                # Append data to the lists.
                artist_followers.append(followers)
                genre.append(first_genre)
                artist_popularity.append(popularity)
            else:
                # If no artist ID, add None to all lists.
                artist_followers.append(None)
                genre.append(None)
                artist_popularity.append(None)
        else:
            # If no artists, add None to all lists.
            artist_followers.append(None)
            genre.append(None)
            artist_popularity.append(None)
    else:
        # If no track, add None to all lists.
        artist_followers.append(None)
        genre.append(None)
        artist_popularity.append(None)

# Step 5: Verifying the lists to check artist information.
print("Artist Followers:", artist_followers)
print("Genre:", genre)
print("Artist Popularity:", artist_popularity)

Artist Followers: [7440, 12884116, 12884116, 32971468, 3346522, 3346522, 3984386, 13054511, 847561, 1038621, 10055696, 10695447, 14613365, 78022, 1352109, 847561, 8830, 41053, 3066834, 5548, 47098853, 1880646, 9244, 3076630, 3076630, 3076630, 54909167, 54107144, 1480988, 249377, 3912887, 4001140, 3984386, 3984386, 5729475, 34383, 15577101, 54107144, 18713236, 1683630, 18713236, 1736232, 18713236, 586286, 506245, 47098853, 43722, 12884116, 166967, 60305, 1571995, 3753569]
Genre: ['lo-fi beats', 'pop', 'pop', 'pop', 'australian pop', 'australian pop', 'desi pop', 'ambient pop', 'sad lo-fi', 'alt z', 'boy band', 'alternative rock', 'dance pop', 'pop r&b', 'sad lo-fi', 'sad lo-fi', None, 'acoustic pop', 'irish rock', 'lo-fi chill', 'emo rap', 'lo-fi chill', 'piano rock', 'irish pop', 'irish pop', 'irish pop', 'modern rock', 'permanent wave', 'acoustic pop', 'acoustic pop', 'pop', 'chill pop', 'desi pop', 'desi pop', 'pop', None, 'modern rock', 'permanent wave', 'pop', 'alt z', 'pop', 'indi

In addition to the above, there's also additional information we can retrieve about each artist. For this purpose, let's first retrieve the list of distinct artists for the different tracks in your playlist.

<div class="alert alert-info"><b>Exercise 8 </b>Write the code to identify the list of unique artist ids that correspond to the different tracks in your chosen playlist. Store these data in a list called <i>unique_artist_ids</i>.</div>

<div class="alert alert-warning">Retrieve the artist ids that correspod directly to the tracks.</div>

In [260]:
# Step 1: Create an empty set to store unique artist IDs.
unique_artist_ids = set()

# Step 2: Loop through the items in the playlist to get artist IDs.
for item in playlist['items']:
    track = item['track']  # Get the track information.

    # Check if the track exists.
    if track:
        artists = track['artists']  # Get the list of artists.

        # Loop through each artist to get their IDs.
        for artist in artists:
            artist_id = artist['id']  # Get the artist ID.
            unique_artist_ids.add(artist_id)  # Add the artist ID to the set.

# Step 3: Convert the set to a list to store unique artist IDs.
unique_artist_ids = list(unique_artist_ids)

# Step 4: Print the list of unique artist IDs.
print("Unique Artist IDs:", unique_artist_ids)

Unique Artist IDs: ['4IWBUUAFIplrNtaOHcJPRM', '2WzaAvm2bBCf4pEhyuDgCY', '3btNpz9NEJ8ml352B8S7Qr', '3QSQFmccmX81fWCUSPTS7y', '4xnihxcoXWK3UqryOSnbw5', '4qIVPF0s71ZYW3qzhu5GkF', '7z6qM92XcUIwugTRs2PcmZ', '4GNC7GD6oZMSxPGyXy4MNB', '4Z8W4fKeB5YxbusRsdQVPb', '3VI0X4zhIZBNzGJtmiBuHc', '4MKVLp1MAwYFqaXhe1g8dA', '35l9BRT7MXmM8bv2WDQiyB', '31TPClRtHm23RisEBtV3X7', '7H55rcKCfwqkyDFH9wpKM6', '4gdMJYnopf2nEUcanAwstx', '7oPftvlwr6VrsViSDV7fJY', '5Sq2960LCdGIniRjmbmT6G', '6ltzsmQQbmdoHHbLZ4ZN25', '14FalyPBwXYg2XLauocHaU', '6bmlMHgSheBauioMgKv2tn', '4gzpq5DPGxSnKTe4SA8HAU', '4BxCuXFJrSWGi1KHcVqaU4', '53XhwfbYqKCa1cC15pYq2q', '1QAJqy2dA3ihHBFIHRphZj', '7qYVVP9RdHRKzElE6EZQsn', '6PKlFeLEuDwKi9jOLf6qWi', '1wxPItEzr7U7rGSMPqZ25r', '2txHhyCwHjUEpJjWrEyqyX', '6ydoSd3N2mwgwBHtF6K7eX', '6KImCVD70vtIoJWnq6nGn3', '0MeLMJJcouYXCymQSHPn8g', '3UAk61T8PItbpgEi9u7ofY', '0MmnmsAuQKRFpo6vJElcaU', '5b0j3TTNSKCByBq4rHYKvG', '3rIZMv9rysU7JkLzEaC5Jp', '15UsOTVnJzReFVN1VCnxy4', '4bm3ooSna3gJa7aj6ZtDka', '1l7ZsJRRS8wlW3WfJ

We are now interested in retrieving catalog information about each artist’s top tracks. This information is provided by Spotify's API on a country basis. Here, we will retrieve the information corresponding to Spain, whose *ISO 3166-1 alpha-2* code is **ES**. The information we are looking for is stored under the ```top-tracks``` endpoint for ```artists```. Requests to this location retrieve the 10 most famous tracks for a given artist id.

<div class="alert alert-info"><b>Exercise 9 </b>Write the code to retrieve the 10 top tracks for each of the unique artists in your chosen playlist. Store these data in a dictionary called <i>top_tracks</i>. The keys of this dictionary should correspond to the unique artist ids stored in the list <i>unique_artist_id</i>. The values of this dictionary should include the names of the 10 most popular songs in a list.</div>

<div class="alert alert-warning">In cases where the number of tracks included in the top-tracks endpoint is below 10, simply retrieve the provided list.</div>


In [261]:
import requests

# Step 1: Create an empty dictionary to store top tracks for each artist.
top_tracks = {}

# Step 2: Define the base URL for the top-tracks endpoint.
top_tracks_endpoint = 'https://api.spotify.com/v1/artists/{id}/top-tracks?market=ES'

# Step 3: Loop through each unique artist ID to retrieve their top tracks.
for artist_id in unique_artist_ids:
    # Step 4: Format the URL with the artist's ID.
    url = top_tracks_endpoint.format(id=artist_id)
    
    # Step 5: Make the GET request to retrieve top tracks for the artist.
    response = requests.get(url, headers=headers)  # Use the headers from the previous steps.
    
    # Step 6: Convert the response to JSON format.
    data = response.json()
    
    # Step 7: Extract track names from the response and store them in the dictionary.
    if 'tracks' in data:
        # Get the names of the top tracks, limited to 10.
        track_names = [track['name'] for track in data['tracks']][:10]
    else:
        track_names = []  # If no tracks are found, store an empty list.
    
    # Step 8: Store the track names in the dictionary with artist ID as the key.
    top_tracks[artist_id] = track_names

# Step 9: Print the top tracks for each artist.
print("Top 10 Tracks for Each Artist:", top_tracks)

Top 10 Tracks for Each Artist: {'4IWBUUAFIplrNtaOHcJPRM': ["Say You Won't Let Go", "Car's Outside", 'Rewrite The Stars (with James Arthur & Anne-Marie)', 'Train Wreck', 'Can I Be Him', 'Impossible', 'Naked', 'A Thousand Years', 'Falling Like The Stars', "Car's Outside - Sped Up Version"], '2WzaAvm2bBCf4pEhyuDgCY': ['Dandelions', 'Lost Boy', 'Dandelions - slowed + reverb', '28 (with Dean Lewis)', 'Superficial Love', 'If By Chance', 'Superficial Love - Single Version', 'do u really?', 'Mixed Signals', '28 with Dean Lewis - Sped Up'], '3btNpz9NEJ8ml352B8S7Qr': ['Yellow Lights', 'Holdin’ On', 'Overwhelmed', 'Just Slide (ft Jaden)', "Can't Let Go (feat. Harry Hudson)", 'Let Me', 'No Good', 'Pendulum', 'Mean To Love', 'A song I wrote after therapy'], '3QSQFmccmX81fWCUSPTS7y': ['Be Alright', 'How Do I Say Goodbye', 'Half A Man', 'Waves', 'Fall At Your Feet (with Dean Lewis)', 'Rest (with Sasha Alex Sloan)', 'Hurtless', '28 (with Dean Lewis)', 'Memories', 'All I Ever Wanted'], '4xnihxcoXWK3Uqr

We can now use this information to identify those songs in your chosen playlist that correspond to each artist's top tracks.

<div class="alert alert-info"><b>Exercise 10 </b>Write the code to check whether the different tracks in your chosen playlist are included among the corresponding artist's top tracks. Store the results in a list called <i>is_top</i>. This list should include the boolean value <i>True</i> whenever the considered track is among the top tracks for that artist and <i>False</i> otherwise.</div>

<div class="alert alert-warning">When comparing track names, consider only <b>exact</b> matches.</div>

In [262]:
is_top = []

for item in playlist['items']:
    track_name = item['track']['name']
    artist_id = item['track']['artists'][0]['id']
    
    # Retrieve the top tracks for the artist from the dictionary
    top_tracks_list = top_tracks.get(artist_id, [])
    
    # Check if the track is in the artist's top tracks
    if track_name in top_tracks_list:
        is_top.append(True)
    else:
        is_top.append(False)

print(is_top)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, False, False, True, True, True, True, False, False, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True]


A very interesting feature in Spotify is the song recommender.

When creating a playlist from scratch, Spotify offers help by suggesting additional tracks that may be related to the ones already added. Spotify's API also provides a specific endpoint for `recommendations`. We will use this endpoint to retrieve one recommendation for each song in our original playlist.

<div class="alert alert-info"><b>Exercise 11 </b>Write the code to retrieve the id, title and artist from the list of recommendations for each track in your playlist. Store these data in new lists called <i>recommendatin_id</i>, <i>recommendation_title</i> and <i>recommendation_artist</i>.</div>

<div class="alert alert-warning">Retrieve recommendations based solely on each track.</div>

<div class="alert alert-warning">Retrieve information only about the first track in the recommendations list.</div>

<div class="alert alert-warning">When more than one artist is associated to the recommended track, retrieve information only about the first.</div>

In [267]:
import requests

# Lists to store recommendation details
recommendation_id = []
recommendation_title = []
recommendation_artist = []

# Loop through each track in the playlist
for item in playlist['items']:
    track_id = item['track']['id']

    # Set the parameters for the recommendations request
    params = {
        'seed_tracks': track_id,
        'limit': 1  # We only want one recommendation 
    }
    
    # Make the request to the recommendations endpoint
    response = requests.get('https://api.spotify.com/v1/recommendations', params=params, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Try to decode the JSON response
            recommendations = response.json().get('tracks', [])
            
            if recommendations:
                first_recommendation = recommendations[0]
                
                # Append the details to the respective lists
                recommendation_id.append(first_recommendation['id'])
                recommendation_title.append(first_recommendation['name'])
                recommendation_artist.append(first_recommendation['artists'][0]['name'])
        except ValueError as e:
            # Handle JSON decoding error
            print(f"JSON decoding error for track ID {track_id}: {e}")
    else:
        print(f"Error fetching recommendations for track ID {track_id}: {response.status_code}")

# Print the lists to verify
print(recommendation_id)
print(recommendation_title)
print(recommendation_artist)

Error fetching recommendations for track ID 3GtXRW9ILD3QCAH3NVdwtA: 429
Error fetching recommendations for track ID 7ce20yLkzuXXLUhzIDoZih: 429
Error fetching recommendations for track ID 60iSKGrGazRzICtMjADNSM: 429
Error fetching recommendations for track ID 1ZMiCix7XSAbfAJlEZWMCp: 429
Error fetching recommendations for track ID 6w8pFOKn42O418qwcQElZ3: 429
Error fetching recommendations for track ID 3EPXxR3ImUwfayaurPi3cm: 429
Error fetching recommendations for track ID 74kCarkFBzXYXNkkYJIsG0: 429
Error fetching recommendations for track ID 0yc6Gst2xkRu0eMLeRMGCX: 429
Error fetching recommendations for track ID 6vJ6FR4WtlKSxe8lmh3F1C: 429
Error fetching recommendations for track ID 06qUEhhx6jKQmhj2qAkn4H: 429
Error fetching recommendations for track ID 1MhXdlCQPnO56T57MfmaRm: 429
Error fetching recommendations for track ID 6b2oQwSGFkzsMtQruIWm2p: 429
Error fetching recommendations for track ID 4rHZZAmHpZrA3iH5zx8frV: 429
Error fetching recommendations for track ID 524dq3zKwXrP4uPL6Vwo

In [264]:
# YOUR CODE HERE

## Bonus exercise

You can complete the exercises below to obtain extra points. These points will be added to your score directly. The maximum score in this assignment is a 10.

<div class="alert alert-danger"><b>Bonus 1 </b>Write the code to find the most popular song in your playlist. If several songs have the same popularity, choose the one for which the artist has the most followers. Save the name of the song to a new variable called <i>most_popular</i>. This variable should <b>only</b> contain the name of the most popular song in string form.</div>

In [265]:
most_popular = None  # Variable to store the name of the most popular song
highest_popularity = -1  # Initialize to a low value

# Step 1: Iterate through the playlist
for i in range(len(title)):
    # Get current song details
    song_title = title[i]
    popularity = artist_popularity[i]  # Song's popularity
    followers = artist_followers[i]  # Artist's followers

    # Step 2: Check if this song is more popular or ties with current highest
    if (popularity > highest_popularity or
        (popularity == highest_popularity and followers > artist_followers[title.index(most_popular)] if most_popular else -1)):
        most_popular = song_title  # Update most popular song
        highest_popularity = popularity  # Update highest popularity

# Step 3: Print the result
print("Most Popular Song:", most_popular)

Most Popular Song: Everglow


You might have noticed that there exist slight incosistencies in the way in which the track title are included in the playlist, versus the top 10 artist tracks. For example, the fourht song in the provided playlist appears as <i>High And Dry</i> within the title of the playlist songs but as <i>High and Dry</i> (with a small "a" for the word "and") in the Radiohead's top tracks. This results in the fact that the song of the playlist will not be included as one of the artist's top tracks in the is_top list.

<div class="alert alert-danger"><b>Bonus 2 </b>Write the code to create a new list called <i>is_top_2</i>. This time make sure that you look for partial matches of title names too.</div>

In [266]:
# Initialize an empty list for the new check
is_top_2 = []

# Loop through each track title in your playlist
for track in title:  # Assuming 'title' is a list of track titles
    # Normalize the track title (lowercase for case insensitivity)
    normalized_track = track.lower().strip()
    
    # Initialize a flag to check if a match is found
    found = False

    # Loop through each unique artist id
    for artist_id in unique_artist_ids:  
        # Get the top tracks for the current artist
        # Ensure 'top_tracks' is a dictionary that maps artist IDs to their top tracks
        if artist_id in top_tracks:
            artist_top_tracks = top_tracks[artist_id]  # Correctly access the artist's top tracks
            
            # Check each top track title
            for top_track in artist_top_tracks:
                normalized_top_track = top_track.lower().strip()  # Normalize the top track title
                
                # Check for partial match
                if normalized_track in normalized_top_track:
                    found = True  # A match is found
                    break  # Exit the top track loop since we found a match

            if found:
                break  # Exit the artist loop since we found a match
    
    # Append the result (True or False) to the is_top_2 list
    is_top_2.append(found)


## Verification and Resolution

# Print the length of the results
print("Length of is_top_2:", len(is_top_2))
print("Length of original playlist:", len(title))

# Print the results for each track
for i, track in enumerate(title):
    print(f"Track: {track}, Is Top Track: {is_top_2[i]}")

top_tracks = {}  # Initialize as a dictionary

for artist_id in unique_artist_ids:
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=ES"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        # Extract track names, handling fewer than 10 tracks gracefully
        track_names = [track['name'] for track in data['tracks'][:10]]
        top_tracks[artist_id] = track_names  # Store in the dictionary
    else:
        print(f"Error retrieving top tracks for artist ID {artist_id}: {response.status_code}")

# Print top tracks for each artist for manual verification
for artist_id in unique_artist_ids:
    print(f"Top tracks for artist ID {artist_id}: {top_tracks[artist_id]}")

Length of is_top_2: 52
Length of original playlist: 52
Track: Tell Me Why I'm Waiting, Is Top Track: True
Track: Before You Go, Is Top Track: True
Track: Hold Me While You Wait, Is Top Track: True
Track: Falling, Is Top Track: True
Track: Waves, Is Top Track: True
Track: Be Alright, Is Top Track: True
Track: Alag Aasmaan, Is Top Track: True
Track: Apocalypse, Is Top Track: True
Track: Baby You're Worth It, Is Top Track: True
Track: Trying My Best, Is Top Track: True
Track: Ghost Of You, Is Top Track: True
Track: Creep, Is Top Track: True
Track: Mirrors, Is Top Track: True
Track: Yellow Lights, Is Top Track: True
Track: death bed (coffee for your head), Is Top Track: True
Track: I've Changed for You, Is Top Track: False
Track: Can't Take My Eyes off You, Is Top Track: True
Track: You & Me (The Wildfire), Is Top Track: True
Track: Chasing Cars, Is Top Track: True
Track: Like U, Is Top Track: True
Track: NUMB, Is Top Track: False
Track: Imagination, Is Top Track: True
Track: I Was Wrong, 